In [4]:
import pandas as pd

prompt_vs_context = pd.read_csv('data/MTBP/Prompt_vs_context/prompt_vs_context.csv')
length_penalty_token = pd.read_csv('data/MTBP/Prompt_vs_context/data_length_penalty_processed.csv')
length_penalty_word= pd.read_csv('data/MTBP/Prompt_vs_context/data_length_penalty_words.csv')
length_penalty_line = pd.read_csv('data/MTBP/Prompt_vs_context/data_length_penalty_lines.csv')


<h1>Length Penalty Algorithm</h1>

Create dataset with alpha-values corresponding:
<p>reference is the length of the context less: </p>
<p></p>
        set the alpha values for all generation step in each problem
                max value equal two, smallest value is zero.five 
                keep only instruction steps codes lenghts 


In [5]:
from get_data import * 
dataset = custom_dataset(prompt_vs_context)

c:\Users\mariu\anaconda3\envs\comp0197-pt\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [37]:
import ast

def generation_functions_with_len_penalty(data, model, early_stopping = None):
    """ 
        Generation of the context functions with a evoluting length penalty.
    """

    # Codes for every problem
    codes_with_context = []

    for j in range(len(data)):
        if early_stopping is not None and j > early_stopping:
            break
        
        # Code for every prompt
        code_with_context = []

        # start with the signature for the incoming problem
        code = data.iloc[j]['signature']

        # initiate the list of prompt to generate
        prompts = ast.literal_eval(data.iloc[j]['prompts'])
        
        # inititate the alphas for the generation penalty
        alphas = data.iloc[j]['alphas']
        
        for i, prompt in enumerate(prompts):
            
            # input text
            function = code + '\n\t#' + prompt

            # tokenization
            ids_function = model.tokenizer.encode(function, return_tensors='pt')

            # generation 
            output_function = model.forward(ids_function)

            # processing 
            processed_output_ids = generation_post_processing(model, output_function)
            
            # length penalty algorithm
            # select within the list of sequence generated the best one according of their log-likelihood
            best_output = pick_best_length(model, processed_output_ids, alphas[i])

            # decoding
            text_function = model.decode_output(best_output)

            # post processing
            text_function = model.post_generation_processing(text_function)

            # enlarge the context
            code = text_function

            # add to the list
            code_with_context.append(text_function)

        codes_with_context.append(code_with_context)

    return codes_with_context

In [6]:
santaCoder = MySantaCoder(max_tokens=128, num_sol=5)

In [7]:
function = 'def function(a):\n\t#Create a list of integers between 0 and a'
ids_function = santaCoder.tokenizer.encode(function, return_tensors='pt')
# generation 
output_function = santaCoder.forward(ids_function)
output_text = generation_post_processing(santaCoder, output_function)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


In [ ]:
codes_with_context = generation_functions_with_len_penalty(dataset, santaCoder, 2)

Exploration of the dataset

In [49]:
import pandas as pd
from post_processing import *

data_len_words = pd.read_csv('data/MTBP/Prompt_vs_context/data_length_penalty_words.csv')
data_len_token = pd.read_csv('data/MTBP/Prompt_vs_context/data_length_penalty_processed.csv')
data_len_lines = pd.read_csv('data/MTBP/Prompt_vs_context/data_length_penalty_lines.csv')
data_ProCo = pd.read_csv('data/MTBP/Prompt_vs_context/prompt_vs_context.csv')

data_len_w = format_testing_lenght_penalty(data_len_words)
data_len_t = format_testing_lenght_penalty(data_len_token)
data_len_l = format_testing_lenght_penalty(data_len_lines)
data_prC = format_testing_prompt_vs_context(data_ProCo)

data_len_w.to_csv('data/MTBP/Prompt_vs_context/words_length_penalty.csv')
data_len_t.to_csv('data/MTBP/Prompt_vs_context/token_length_penalty.csv')
data_len_l.to_csv('data/MTBP/Prompt_vs_context/lines_lenght_penalty.csv')
data_prC.to_csv('data/MTBP/Prompt_vs_context/prompt_vs_context.csv')